# refellips demo: Solid/Liquid
This demo presents an example use case for fitting multi-wavelength single-angle ellipsometry data for solid-liquid systems with refellips.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from refnx.analysis import CurveFitter
from refnx.reflect import Slab

In [3]:
from refellips.dataSE import DataSE, open_EP4file
from refellips import ObjectiveSE, RI, Cauchy, load_material, ReflectModelSE

### Import data

In [ ]:
dname = 'WVASE_example_2nmSiO2_100nmPNIPAM_MultiWavelength_Water.dat'
data = DataSE(data=dname)

### Build model

In [ ]:
from refnx.reflect import Linear, Erf

si = load_material("silicon")
sio2 = load_material("silica")
PNIPAM = load_material("pnipam")
h2o = load_material("water")

PNIPAM_layer = PNIPAM(1000)
PNIPAM_layer.thick.setp(vary=True, bounds=(100,1500))
PNIPAM_layer.vfsolv.setp(vary=True, value=0.8, bounds=(0.1,1))
PNIPAM_layer.rough.setp(vary=True, value=100, bounds=(50,300))
PNIPAM_layer.interfaces = Erf()

struc = h2o() | PNIPAM_layer | sio2(20) | si()
struc.contract = 2
struc.solvent= h2o

model = ReflectModelSE(struc)
model.delta_offset.setp(value=0, vary=True, bounds=(-10,10))

# Fitting experimental data

### Model & data pre fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

wavelength_aois = np.c_[data.wavelength, data.aoi]
psi, delta = model(wavelength_aois)

ax.scatter(data.wavelength, data.psi, color='r')
axt.scatter(data.wavelength, data.delta, color='b')

ax.plot(data.wavelength, psi, color='r')
axt.plot(data.wavelength, delta, color='b')

ax.set(ylabel='Psi', xlabel='Wavelength (nm)')
axt.set(ylabel='Delta')

In [ ]:
objective = ObjectiveSE(model, data)

In [ ]:
fitter = CurveFitter(objective)
fitter.fit(method='least_squares');

### Model & data post fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

wavelength_aois = np.c_[data.wavelength, data.aoi]
psi, delta = model(wavelength_aois)

p = ax.scatter(data.wavelength, data.psi, color='r')
d = axt.scatter(data.wavelength, data.delta, color='b')

ax.plot(data.wavelength, psi, color='r')
axt.plot(data.wavelength, delta, color='b')

ax.legend(handles=[p,d], labels=['Psi', 'Delta'])
ax.set(ylabel='Psi', xlabel='Wavelength, nm')
axt.set(ylabel='Delta')

### Refractive index profile

In [ ]:
fig, ax = plt.subplots()
struc.reverse_structure = True
ax.plot(*struc.ri_profile())
struc.reverse_structure = False
ax.set_ybound(1.3,1.6)
ax.set(ylabel='Refractive index', xlabel='Distance from substrate (Å)')

In [ ]:
print(objective)